ELEC6910X Advanced Topics in AI and Healthcare

# Assignment 1

***Holy Lovenia - 20814158***

------

In [1]:
from collections import OrderedDict
from PIL import Image
from pytorch3dunet.unet3d import losses
from torch import nn
from torch.autograd import Variable
from torch.nn import init
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomVerticalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)
from transformers import AutoConfig, AutoFeatureExtractor, AutoModelForImageClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from tqdm import tqdm

import datasets
import h5py
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import surface_distance.metrics as sf
import torch
import torch.nn.functional as F
import torch.optim as optim
import torchio as tio
import torchmetrics
import torchvision

In [2]:
RANDOM_SEED = 42

def set_all_seeds(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

set_all_seeds(RANDOM_SEED)

## Problem 2

In [3]:
# model_name_or_path = "microsoft/resnet-152"
# model_name_or_path = "gianlab/swin-tiny-patch4-window7-224-finetuned-skin-cancer"
model_name_or_path = "google/vit-base-patch16-224-in21k"

In [4]:
metric = datasets.load_metric("accuracy")

/tmp/ipykernel_3226/3770023701.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("accuracy")


In [5]:
def problem2_load_dataset(manifest_path, img_dir_path, img_column_name="image_train", label_column_name="label_train"):
    manifest_df = pd.read_csv(manifest_path)
    manifest_df[img_column_name] = img_dir_path + manifest_df[img_column_name].astype(str)
    manifest_df = manifest_df.rename(columns={img_column_name: "image", label_column_name: "label"})
    dataset = datasets.Dataset.from_pandas(manifest_df)
    dataset = dataset.cast_column("image", datasets.Image(decode=True))
    dataset = dataset.class_encode_column("label")
    return dataset

In [6]:
def ham10000_load_dataset(manifest_path, img_dir_paths, img_column_name="image_id", label_column_name="dx"):
    manifest_df = pd.read_csv(manifest_path)
    for i in range(len(manifest_df)):
        for img_dir_path in img_dir_paths:
            img_path = os.path.join(img_dir_path, f"{manifest_df[img_column_name][i]}.jpg")
            if os.path.isfile(img_path):
                manifest_df[img_column_name][i] = img_path
    manifest_df = manifest_df.drop(manifest_df[~manifest_df[img_column_name].str.contains(".jpg")].index)
    manifest_df = manifest_df.rename(columns={img_column_name: "image", label_column_name: "label"})
    manifest_df = manifest_df.replace({
        "mel": 0,
        "nv": 1,
        "bcc": 2,
        "akiec": 3,
        "bkl": 4,
        "df": 5,
        "vasc": 6,
    })
    dataset = datasets.Dataset.from_pandas(manifest_df)
    dataset = dataset.cast_column("image", datasets.Image(decode=True))
    dataset = dataset.class_encode_column("label")
    return dataset

In [7]:
raw_datasets = datasets.DatasetDict()
raw_datasets["train"] = problem2_load_dataset("./data/problem2_datas/annotation/train.csv", "./data/problem2_datas/images/")
# train_val_dataset = problem2_load_dataset("./data/problem2_datas/annotation/train.csv", "./data/problem2_datas/images/").train_test_split(test_size=0.2)
# raw_datasets["train"] = train_val_dataset["train"]
# raw_datasets["valid"] = train_val_dataset["test"]
raw_datasets["test"] = problem2_load_dataset("./data/problem2_datas/annotation/test.csv", "./data/problem2_datas/images/")

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
pretrain_datasets = datasets.DatasetDict()
pretrain_train_val_dataset = ham10000_load_dataset(
    "./data/HAM10000/HAM10000_metadata.csv",
    [
        "./data/HAM10000/ham10000_images_part_1",
        "./data/HAM10000/HAM10000_images_part_1",
        "./data/HAM10000/ham10000_images_part_2",
        "./data/HAM10000/HAM10000_images_part_2"
    ]).train_test_split(test_size=0.2)
pretrain_datasets["train"] = pretrain_train_val_dataset["train"]
pretrain_datasets["valid"] = pretrain_train_val_dataset["test"]

/tmp/ipykernel_3226/4003405763.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  manifest_df[img_column_name][i] = img_path


Stringifying the column:   0%|          | 0/11 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/11 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

In [9]:
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name_or_path)
model = AutoModelForImageClassification.from_pretrained(model_name_or_path, num_labels=7)

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
normalize = Normalize(mean=feature_extractor.image_mean, std=feature_extractor.image_std)
train_transforms = Compose(
        [
#             RandomResizedCrop(feature_extractor.size),
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            RandomHorizontalFlip(),
            RandomVerticalFlip(),
            ToTensor(),
            normalize,
        ]
    )

eval_transforms = Compose(
        [
            Resize(feature_extractor.size),
            CenterCrop(feature_extractor.size),
            ToTensor(),
            normalize,
        ]
    )

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_eval(example_batch):
    """Apply eval_transforms across a batch."""
    example_batch["pixel_values"] = [
        eval_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

In [11]:
proc_datasets = raw_datasets
proc_datasets["train"].set_transform(preprocess_train)
# proc_datasets["valid"].set_transform(preprocess_eval)
proc_datasets["test"].set_transform(preprocess_eval)

In [12]:
proc_pretrain_datasets = pretrain_datasets
proc_pretrain_datasets["train"].set_transform(preprocess_train)
proc_pretrain_datasets["valid"].set_transform(preprocess_eval)

In [13]:
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [14]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

#### Pretrain

In [15]:
args = TrainingArguments(
    f"save/problem2/pretrain__{model_name_or_path}__1e-3__weighted",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=200,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    gradient_checkpointing=True,
    save_total_limit=1,
    seed=RANDOM_SEED,
    data_seed=RANDOM_SEED,
    dataloader_num_workers=4,
    lr_scheduler_type="cosine",
    fp16=True,
)

In [16]:
class Problem2PretrainingTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to("cuda")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits").to("cuda")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.11, 0.68, 0.05, 0.03, 0.11, 0.01, 0.01])).to("cuda")
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [17]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
trainer = Problem2PretrainingTrainer(
    model,
    args,
    train_dataset=proc_pretrain_datasets["train"],
    eval_dataset=proc_pretrain_datasets["valid"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],
)

Using cuda_amp half precision backend


In [18]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/home/holy/anaconda3/envs/health/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1600
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 2400


Epoch,Training Loss,Validation Loss,Accuracy
0,1.767000,1.316271,0.567500
1,1.221100,0.762621,0.705000
2,0.777100,0.623710,0.742500
3,0.575900,0.473759,0.812500
4,0.397600,0.494391,0.787500
5,0.393200,0.401802,0.827500
6,0.572500,0.360019,0.825000
7,0.239300,0.431710,0.815000
8,0.252300,0.668098,0.777500
9,0.239000,0.583459,0.782500


***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-12
Configuration saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-12/config.json
Model weights saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-12/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-12/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-24
Configuration saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-24/config.json
Model weights saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-24/pyto

Feature extractor saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-144/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-156
Configuration saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-156/config.json
Model weights saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-156/pytorch_model.bin
Feature extractor saved in save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-156/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/checkpoint-144] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32
Saving model checkpoint to save/problem2/pretrain__google/vit-

***** train metrics *****
  epoch                    =        20.96
  total_flos               = 2425021988GF
  train_loss               =       0.4129
  train_runtime            =   0:07:23.71
  train_samples_per_second =      721.186
  train_steps_per_second   =        5.409


In [19]:
metrics = trainer.evaluate(proc_datasets["test"])
# some nice to haves:
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

***** Running Evaluation *****
  Num examples = 99
  Batch size = 32


***** test metrics *****
  epoch                   =      20.96
  eval_accuracy           =     0.1818
  eval_loss               =     4.8261
  eval_runtime            = 0:00:01.79
  eval_samples_per_second =     55.191
  eval_steps_per_second   =       2.23


In [20]:
metrics = trainer.evaluate(proc_pretrain_datasets["valid"])
# some nice to haves:
trainer.log_metrics("valid", metrics)
trainer.save_metrics("valid", metrics)

***** Running Evaluation *****
  Num examples = 400
  Batch size = 32


***** valid metrics *****
  epoch                   =      20.96
  eval_accuracy           =       0.86
  eval_loss               =     0.2888
  eval_runtime            = 0:00:02.68
  eval_samples_per_second =    148.984
  eval_steps_per_second   =      4.842


#### Problem 2 dataset

In [21]:
model = AutoModelForImageClassification.from_pretrained(f"save/problem2/pretrain__{model_name_or_path}__1e-3__weighted")
# model = AutoModelForImageClassification.from_pretrained(model_name_or_path)

loading configuration file save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted/config.json
Model config ViTConfig {
  "_name_or_path": "save/problem2/pretrain__google/vit-base-patch16-224-in21k__1e-3__weighted",
  "architectures": [
    "ViTForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "image_size": 224,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "problem_type": "single_label_class

In [22]:
args = TrainingArguments(
    f"save/problem2/pretrained__1e-3__pretrained|{model_name_or_path}__1e-3",
#     f"save/problem2/{model_name_or_path}__5e-4__cwr",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=200,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    gradient_checkpointing=True,
    save_total_limit=1,
    seed=RANDOM_SEED,
    data_seed=RANDOM_SEED,
    dataloader_num_workers=4,
    lr_scheduler_type="cosine",
    fp16=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [23]:
class Problem2Trainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to("cuda")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits").to("cuda")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.11, 0.68, 0.05, 0.03, 0.11, 0.01, 0.01])).to("cuda")
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [24]:
trainer = Problem2Trainer(
    model,
    args,
    train_dataset=proc_datasets["train"],
    eval_dataset=proc_datasets["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=20)],
)

Using cuda_amp half precision backend


In [25]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/home/holy/anaconda3/envs/health/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 305
  Num Epochs = 200
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 400


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,4.334931,0.181818
1,No log,1.588728,0.494949
2,No log,0.667072,0.676768
3,No log,0.485098,0.676768
4,2.456800,0.383083,0.676768
5,2.456800,0.334649,0.676768
6,2.456800,0.303307,0.676768
7,2.456800,0.276993,0.676768
8,2.456800,0.244997,0.676768
9,0.384900,0.227124,0.696970


***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-2
Configuration saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-2/config.json
Model weights saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-2/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-2/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-4
Configuration saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-4/config.json
Model weights saved in save/problem2/pretrained__1e-3__pretrained|goo

***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-24
Configuration saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-24/config.json
Model weights saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-24/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-24/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-26
Configuration saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-26/config.json
Model weights saved in save/problem2/pretrained__1e-3__pretrain

***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-46
Configuration saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-46/config.json
Model weights saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-46/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-46/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-44] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-48
Configuration saved in save/problem2/pret

Deleting older checkpoint [save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-64] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving model checkpoint to save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-68
Configuration saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-68/config.json
Model weights saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-68/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-68/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-66] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 99
  Batch size = 32
Saving mode

Feature extractor saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-88/preprocessor_config.json
Deleting older checkpoint [save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-86] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/checkpoint-48 (score: 0.7676767676767676).
Saving model checkpoint to save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3
Configuration saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/config.json
Model weights saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3/pytorch_model.bin
Feature extractor saved in save/problem2/pretrained__1e-3__pretrained|google/vit-base-patch16-224-in21k__1e-3

***** train metrics *****
  epoch                    =        43.8
  total_flos               = 968565329GF
  train_loss               =      0.4576
  train_runtime            =  0:05:10.21
  train_samples_per_second =     196.639
  train_steps_per_second   =       1.289


In [26]:
metrics = trainer.evaluate(proc_datasets["test"])
# some nice to haves:
trainer.log_metrics("test", metrics)
trainer.save_metrics("test", metrics)

***** Running Evaluation *****
  Num examples = 99
  Batch size = 32


***** test metrics *****
  epoch                   =       43.8
  eval_accuracy           =     0.7677
  eval_loss               =     0.1721
  eval_runtime            = 0:00:01.80
  eval_samples_per_second =     54.778
  eval_steps_per_second   =      2.213


In [27]:
# metrics = trainer.evaluate(proc_datasets["valid"])
# # some nice to haves:
# trainer.log_metrics("valid", metrics)
# trainer.save_metrics("valid", metrics)

In [28]:
metrics = trainer.evaluate(proc_datasets["train"])
# some nice to haves:
trainer.log_metrics("train", metrics)
trainer.save_metrics("train", metrics)

***** Running Evaluation *****
  Num examples = 305
  Batch size = 32


***** train metrics *****
  epoch                   =       43.8
  eval_accuracy           =     0.8131
  eval_loss               =     0.1216
  eval_runtime            = 0:00:02.50
  eval_samples_per_second =    121.841
  eval_steps_per_second   =      3.995
